In [1]:
import json
import requests

from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from typing import List, Optional

class ChatGLM2(LLM):
    max_token: int = 2048
    temperature: float = 0.1
    top_p = 0.7
    history = []
    
    def __init__(self, temperature=temperature):
        super().__init__()
        self.temperature = temperature
        
    @property
    def _llm_type(self) -> str:
        return "ChatGLM2"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {'Content-Type': 'application/json'}
        data = json.dumps({
            'prompt':prompt,
            'temperature':self.temperature, 
            'history':self.history,
            'max_length':self.max_token})
        print("ChatGLM prompt:", prompt)
        # 调用api 
        response = requests.post("http://192.168.16.150:8000", headers=headers, data=data, timeout=30)
        if response.status_code!=200:
            return "查询结果错误"
        resp = response.json()
        if stop is not None:
            response = enforce_stop_tokens(response, stop)
        self.history = self.history + [[None, resp['response']]]
        return resp['response']

In [2]:
from langchain import PromptTemplate

llm = ChatGLM2(temperature=0.1)

In [5]:
template = """
    内容生成任务：请根据下面的提示，生成一段文本。
    
    {text}
"""
prompt = PromptTemplate(
   input_variables=["text"],
   template=template,
)
text = '小明老婆昨天生了一对龙凤胎，他们还有一个4岁的女儿，那么现在小明一共有几个孩子？分别是几个儿子，几个女儿？'
prompt_format = prompt.format(text=text)
output = llm.invoke(prompt_format)
print(output)

ChatGLM prompt: 
    内容生成任务：请根据下面的提示，生成一段文本。
    
    小明老婆昨天生了一对龙凤胎，他们还有一个4岁的女儿，那么现在小明一共有几个孩子？分别是几个儿子，几个女儿？

小明老婆昨天生了一对龙凤胎，他们还有一个4岁的女儿。所以，小明现在一共有三个孩子。其中，两个是儿子，一个女儿。


In [3]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
messages = [
    AIMessage(content="Hi."),
    SystemMessage(content="Your role is a program coder."),
    HumanMessage(content="小明老婆昨天生了一对龙凤胎，他们还有一个4岁的女儿，那么现在小明一共有几个孩子？分别是几个儿子，几个女儿？"),
]
output = llm.invoke(messages)
print(output)

ChatGLM prompt: AI: Hi.
System: Your role is a program coder.
Human: 小明老婆昨天生了一对龙凤胎，他们还有一个4岁的女儿，那么现在小明一共有几个孩子？分别是几个儿子，几个女儿？
AI: 现在小明一共有三个孩子。分别是两个儿子和一个女儿。
